In [49]:
import carla
import time
import numpy as np
import keyboard
import cv2
import random


# Connect to the CARLA simulator
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()



In [47]:
carla = client.load_world('Town01')

In [52]:
# Obtention des coordonnées de la caméra
spectator = world.get_spectator()
camera = world.get_spectator()
camera_transform = camera.get_transform()
camera_location = camera_transform.location
camera_rotation = camera_transform.rotation
print(f"Camera location: {camera_location}")
print(f"Camera rotation: {camera_rotation}")

RuntimeError: internal error: unable to find spectator

In [50]:
# Tentative de faire créer un long trajet pour faire un dataset valide
# Concluion : Très long à faire pour le peu d'image obtenu
def generation_trajet():
    blueprint = world.get_blueprint_library().find("vehicle.tesla.model3")
    spawn_point = carla.Transform(carla.Location(x=330.845581, y=195.527863, z=3), carla.Rotation(pitch=0.000000, yaw=180.000000, roll=0.000000))
    vehicle = world.spawn_actor(blueprint, spawn_point)

    blueprint = world.get_blueprint_library().find("vehicle.audi.a2")
    spawn_point = carla.Transform(carla.Location(x=309.853577, y=199.227951, z=3))
    vehicle2 = world.spawn_actor(blueprint, spawn_point)

    blueprint = random.choice(world.get_blueprint_library().filter('vehicle'))
    spawn_point = carla.Transform(carla.Location(x=270.685883, y=198.910965, z=3))
    vehicle3 = world.spawn_actor(blueprint, spawn_point)

    blueprint = random.choice(world.get_blueprint_library().filter('vehicle'))
    spawn_point = carla.Transform(carla.Location(x=245.644699, y=199.270432, z=3))
    vehicle4 = world.spawn_actor(blueprint, spawn_point)

    blueprint = random.choice(world.get_blueprint_library().filter('vehicle'))
    spawn_point = carla.Transform(carla.Location(x=197.804962, y=199.209579, z=3))
    vehicle5 = world.spawn_actor(blueprint, spawn_point)

    blueprint = random.choice(world.get_blueprint_library().filter('walker'))
    spawn_point = carla.Transform(carla.Location(x=306.125458, y=191.453323, z=3))
    pieton1 = world.spawn_actor(blueprint, spawn_point)

    blueprint = random.choice(world.get_blueprint_library().filter('walker'))
    spawn_point = carla.Transform(carla.Location(x=285.016296, y=191.565521, z=3))
    pieton2 = world.spawn_actor(blueprint, spawn_point)



generation_trajet()

In [51]:
for v in world.get_actors():
    v.destroy()

In [2]:
# Fonction créer qui permet de changer de météo afin d'améliorer notre modèle de réseau de neurones dans le futur
weatherList = np.array([carla.WeatherParameters.WetCloudySunset,carla.WeatherParameters.CloudyNoon,carla.WeatherParameters.WetNoon,carla.WeatherParameters.WetCloudyNoon,carla.WeatherParameters.MidRainyNoon,carla.WeatherParameters.HardRainNoon,carla.WeatherParameters.SoftRainNoon,carla.WeatherParameters.ClearSunset,carla.WeatherParameters.CloudySunset,carla.WeatherParameters.WetSunset,carla.WeatherParameters.MidRainSunset,carla.WeatherParameters.HardRainSunset,carla.WeatherParameters.SoftRainSunset])
np.random.shuffle(weatherList)
world.set_weather(weatherList[0])

In [3]:
weather_parameters = {
    'ClearNoon': carla.WeatherParameters.ClearNoon,
    'WetCloudySunset': carla.WeatherParameters.WetCloudySunset,
    'SoftRainNoon': carla.WeatherParameters.SoftRainNoon,
    'MidRainSunset': carla.WeatherParameters.MidRainSunset,
    'HardRainSunset': carla.WeatherParameters.HardRainSunset,
    'ClearSunset': carla.WeatherParameters.ClearSunset,
    'WetSunset': carla.WeatherParameters.WetSunset,
    'SoftRainSunset': carla.WeatherParameters.SoftRainSunset
}
random_key = random.choice(list(weather_parameters.keys()))
random_value = weather_parameters[random_key]
world.set_weather(random_value)

In [2]:
# Charger le véhicule principal
def charger_vehicle():
    bp_lib = world.get_blueprint_library()
    spawn_points = world.get_map().get_spawn_points()
    vehicule_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
    vehicle =world.try_spawn_actor(vehicule_bp,random.choice(spawn_points))
    return vehicle

In [3]:
# Faire spawner plusieurs voiture dans le monde 
def spanwnVehicle(n):
    for i in range (n):
        vehicle_bp = random.choice(bp_lib.filter('vehicle'))
        npc = world.try_spawn_actor(vehicle_bp,random.choice(spawn_points))


In [ ]:
# Faire supprimer tous les véhicules du monde
def destruction():
    for v in world.get_actors().filter('*vehicle*'):
        v.destroy()

In [4]:
def isCirculation(value):
    for v in world.get_actors().filter('*vehicle*'):
        v.set_autopilot(value)


In [5]:
def attach_cam(vehicle):
    # Génération d'une caméra RGB
    camera = world.get_blueprint_library().find("sensor.camera.rgb")
    camera.set_attribute("image_size_x", str(800))
    camera.set_attribute("image_size_y", str(600))

    # Génération d'une caméra sémantique
    cameraS = world.get_blueprint_library().find("sensor.camera.semantic_segmentation")
    cameraS.set_attribute("image_size_x", str(800))
    cameraS.set_attribute("image_size_y", str(600))

    # Attacher les caméras au véhicule
    spawn_point = carla.Transform(carla.Location(z=2))
    sensor = world.spawn_actor(camera, spawn_point, attach_to=vehicle)
    sensorS = world.spawn_actor(cameraS, spawn_point, attach_to=vehicle)
    return sensor,sensorS

In [9]:
# Fonction qui permet de récupérer les images des caméras
def process_img(image,typeCam,weather):
    time.sleep(2)
    # Si on souhaite avoir des images de segmentation selon la palette de couleurs de CityScapes ( évite d'avoir une image noir car l'image contient les numéros des classes )
   # if(typeCam == "seg"):
   #      image.convert(carla.ColorConverter.CityScapesPalette)
    # Convert the image to a numpy array
    np_image = np.array(image.raw_data)
    np_image = np_image.reshape((image.height, image.width, 4))

    # Compteurs globaux pour les images RGB et Segmentation
    global counterRGB
    counterRGB += 1
    
    global counterSEG
    counterSEG += 1

    #Enregistrement des images RGB et Segmentation
    if(typeCam == "rgb" ):
        filename = "dataset/"+weather+"/rgb/image{}.png".format(counterRGB)
        filenameAll = "dataset/All_Image/rgb/image{}.png".format(counterRGB)
        cv2.imwrite(filename, np_image)
        cv2.imwrite(filenameAll,np_image)
    elif(typeCam == "seg"):
        filename = "dataset/"+weather+"/segmentation/image{}.png".format(counterSEG)
        filenameAll = "dataset/All_Image/segmentation/image{}.png".format(counterSEG)
        cv2.imwrite(filename, np_image)
        cv2.imwrite(filenameAll,np_image)


In [10]:
def dataset():
    weather_parameters = {
    'ClearNoon': carla.WeatherParameters.ClearNoon,
    'WetCloudySunset': carla.WeatherParameters.WetCloudySunset,
    'SoftRainNoon': carla.WeatherParameters.SoftRainNoon,
    'MidRainSunset': carla.WeatherParameters.MidRainSunset,
    'HardRainSunset': carla.WeatherParameters.HardRainSunset,
    'ClearSunset': carla.WeatherParameters.ClearSunset,
    'WetSunset': carla.WeatherParameters.WetSunset,
    'SoftRainSunset': carla.WeatherParameters.SoftRainSunset
    }
    list_weather = list(weather_parameters.keys())
    for weather in list_weather :
        params = weather_parameters[weather]
        world.set_weather(params)
        vehicle = charger_vehicle()
        spanwnVehicle(15)
        isCirculation(True)
        sensor,sensorS = attach_cam(vehicle)

        # Enregistrement des images
        sensor.listen(lambda im:process_img(im,"rgb",weather))
        sensorS.listen(lambda im:process_img(im,"seg",weather))

        # Lance la simulation pendant 60 secondes
        time.sleep(60)
        # Arrête la simulation, arrête l'enregistrement des images et détruit les véhicules
        sensor.stop()
        sensorS.stop()

        destruction()


bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
# Initialisation des compteurs
counterRGB = 0
counterSEG = 0

dataset()
